In [35]:
"""This code generates patient summary for doctor from the input symptoms"""

import requests
import json
import os
from IPython.display import display, Markdown
import re

    
# Function to create the system prompt with examples
def create_system_prompt(examples):
    # Initialize the system prompt
    system_prompt = """
    You are an experienced medical scribe. Your task is to read the patient's input and compose a professional, readable, and concise patient summary 
    in narrative form, suitable for a doctor's review, without French lines, bullet points, or parentheses. Ensure that the summary includes the patient's age, gender, relevant medical history, current 
    medications, chief complaint, history of present illness, pertinent positives and negatives, social history, and any additional information or 
    requests provided by the patient. Write the summary in the third person, maintaining a professional and clinical tone. Stick to the conditions 
    mentioned in the input. Do not invent anything. Additionally, follow these rules:
    1. Create a heading "Result:" followed by the reason for contact. Enclose "Result:" in double stars.
    2. The second heading is "Answers to the preliminary survey:" (also enclosed in double stars), followed by a narrative description of the patient's age, underlying diseases, and medications in the first paragraph. If there are no underlying diseases or medications, or if the client does not know that he/she has them, mention this at this stage. For example, if “basic diseases” are mentioned in the section “The customer states in the preliminary information survey that he/she does not have:”, then write "No basic diseases" after the age. 
       After that, on a new line, list the conditions mentioned under the heading “The customer states in the preliminary information survey that he/she has:”. When editing the text, use the past tense as needed.
    3. For any statements in the first person inside parentheses (e.g., "I measured 38.5°C yesterday"), rephrase them into the third person, attributing them to the client (e.g., "The client reports measuring 38.5°C yesterday"). Remove the parentheses in the final text.
    4. If the client has performed an action, such as measuring temperature, rephrase it to "the client reports" or "the client's measurement".
    5. If the preliminary information includes the client's needs or wishes regarding contact, write this briefly on its own line after the symptoms. If the need or wish is not mentioned in the preliminary information, do not invent it; in that case, leave this paragraph unwritten. 
    6. After these, create a new paragraph listing the items under “The customer states in the preliminary information survey that he/she does not have:”. Phrase these as "No [symptom]" or "Does not have [condition]", adjusting the wording as necessary.
    7. If the text does not contain at least one of the following sentences in Finnish, Swedish, or English, then do nothing but apologize and state that you can only modify the texts of the preliminary information survey and cannot proceed further.
    In Finnish:    
    "Asiakas kertoo ennakkotietotutkimuksessa, että hänellä on:"
    "Asiakas ilmoittaa ennakkotietokyselyssä, ettei hänellä ole:"
    "Asiakas ei ennakkotietotutkimuksessa tiedä, että hänellä on:"
    
    In English:
    "In the Symptom questionnaire, the client tells that he/she has:"
    "In the Symptom questionnaire, the client tells that he/she does not have:"
    "In the Symptom questionnaire, the client does not know if he/she has:"
    
    In Swedish:
    "I frågeformuläret uppger klienten att hen har:"
    "I frågeformuläret uppger klienten att hen inte har:"
    "I frågeformuläret vet klienten inte om hen har:"

    Here are some examples of input symptoms and corresponding summary. Strictly follow their format:
    """
    
    # Loop through each example and append input and summary in the prompt
    for i, example in enumerate(examples, 1):
        input_symptoms = example[0]  # The input section
        summary = example[1]         # The summary section
        system_prompt += f"\nExample {i}:\nInput Symptoms: {input_symptoms}\nSummary: {summary}\n"
    
    return system_prompt


# Function to send a request to GPT and generate a summary
def generate_medical_summary(patient_input, examples):
    # Construct the system prompt with the given examples
    system_prompt = create_system_prompt(examples)

    input_prompt = f"Input Symptoms: {patient_input}"

    api_key = os.getenv("OPENAI_KEY")  # Ensure the environment variable is set
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    payload = {
        "model": "gpt-4o-2024-08-06",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": input_prompt}
        ],
        "max_tokens": 500,
        "temperature": 0
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # Parse the response and return the generated summary
    if response.status_code == 200:
        response_data = response.json()
        output = response_data['choices'][0]['message']['content']
        return output
    else:
        print(f"Request failed with status code {response.status_code}")
        print("Response Body:", response.text)
        return None

# Example input-output pairs (taken from the provided data)
examples = [
    [
        """
        A 33-year-old man.
        Reason for contact: sore throat.
        The customer states in the preliminary information survey that he has:
        - basic diseases (asthma)
        - using regular medication (Flixotide in the morning and evening)
        - sore throat (a couple of days, the pain gets worse slowly)
        - fever (I measured 38.5C yesterday and 38C today)
        - cough
        - runny nose
        - the client has used treatment methods (paracetamol and a gun help with pain and fever)
        - the customer feels the need for sick leave (I can't go to work at the store's checkout right now)
        - the customer also tells us about the situation or the need (prescription for fever medicine and sick leave. Should I go to the reception to show my throat?)
        
        The customer states in the preliminary information survey that he does not have:
        - difficulty swallowing food or drink
        - white coatings in the mouth or pharynx
        - difficulty opening or closing the mouth
        - feeling of pressure in the face
        - earache
        - headache
        - shortness of breath
        - chest pain
        - the condition has deteriorated so much that the client would consider going to the emergency room
    
        In the preliminary information survey, the customer does not know that he has:
        - very strong sore throat
        """,
        
        """
        **Result:** sore throat
        **Answers to the preliminary survey:**
        
        33-year-old man. Basic diseases: asthma. Regular medication: Flixotide in the morning and evening.
        
        The client has had a sore throat for a couple of days, with the pain getting worse slowly. He reports measuring 38.5°C yesterday and 38°C today. He also has a cough and runny nose. He has used paracetamol and sauna, which help with pain and fever.
        
        He feels the need for sick leave as he cannot work at the store's checkout right now. He requests a prescription for fever medicine and sick leave, and asks if he should come in to have his throat examined.
        
        No difficulty swallowing food or drink, no white coatings in the mouth or pharynx, no difficulty opening or closing the mouth, no feeling of pressure in the face, no earache, no headache, no shortness of breath, no chest pain, and his condition has not deteriorated to the extent that he would consider going to the emergency room.
        
        The client does not know if he has a very strong sore throat.
        """
    ],
    [
        """
        A 44-year-old man.
        Reason for contact: heartburn.
        The customer states in the preliminary information survey that he has:
        - less than 1 day of heartburn
        - abdominal pain (the last time it was in the middle of the upper abdomen two months ago and now again)
        - the customer has used treatment methods (some over-the-counter acid medicine from the pharmacy that does not seem to help)
        - the client also tells about the situation or the need (heartburn gone. should I be examined when this happened again?)
    
        The customer states in the preliminary information survey that he does not have:
        - basic diseases
        - using regular medication
        - surgeries in the abdominal area
        - reflux
        - chest pain
        - nausea
        - vomiting
        - blood in stool or when wiping
        - black stools
        - very strong stomach pain
        - shortness of breath
        - The condition has deteriorated so much that the customer would consider going to the emergency room
        """,
    
        """
        **Result:** heartburn

        **Answers to the preliminary survey:**
        
        44-year-old man. No basic diseases. No regular medication.
        
        The client has had less than one day of heartburn. He reports abdominal pain; the last time it was in the middle of the upper abdomen two months ago and now again. He has used some over-the-counter acid medicine from the pharmacy that does not seem to help.
        
        He notes that the heartburn is gone and asks whether he should be examined since this happened again.
        
        No surgeries in the abdominal area, no reflux, no chest pain, no nausea, no vomiting, no blood in stool or when wiping, no black stools, no very strong stomach pain, no shortness of breath, and his condition has not deteriorated to the extent that he would consider going to the emergency room.
        
        """
    ]
]

# Sample patient input (taken from the provided examples)
patient_input =     """
        A 27-year-old man
        Reason for contact: distressing
        
        The customer states in the preliminary information survey that he has:
        - basic diseases (hypothyroidism)
        -when using regular medication ( Thyroxin )
        -anxiety (always had it all my life from time to time, but now it's been difficult for a week)
        - stress
        -concentration difficulties (I can't really study)
        -difficulty starting or completing things (I can't get tasks done)
        -fatigue
        -difficulties sleeping (I wake up anxious sometimes)
        -symptoms of depression (a little at times)
        -alcohol consumption (about once a month 10 doses at a party)
        -the client has used treatment methods (I tried online anxiety therapy at a mental health center, it didn't help)
        - the customer feels the need for sick leave (I am not allowed to study)
        - the customer also tells about the situation or need (help with this anxiety)
        
        The customer states in the preliminary information survey that he does not have:
        - thoughts that are dangerous for oneself or others
        - dangerous behavior for oneself or others
        - shortness of breath
        - the use of substances
        -the condition has deteriorated so much that the customer would consider going to the emergency room

    """
# Generate the summary based on patient input and examples
summary = generate_medical_summary(patient_input, examples)

# Output the generated summary
if summary:
    # Display the formatted summary as Markdown
    display(Markdown(formatted_summary))


**Result:** distressing

**Answers to the preliminary survey:**

27-year-old man. Basic diseases: hypothyroidism. Regular medication: Thyroxin.

The client has experienced anxiety, which he has had all his life from time to time, but it has been particularly difficult for the past week. He reports stress, concentration difficulties, inability to start or complete tasks, fatigue, difficulties sleeping with occasional anxious awakenings, and symptoms of depression at times. He consumes alcohol about once a month, typically 10 doses at a party. He has tried online anxiety therapy at a mental health center, which did not help. He feels the need for sick leave as he is unable to study and requests help with his anxiety.

No thoughts or behavior dangerous to oneself or others, no shortness of breath, no use of substances, and his condition has not deteriorated to the extent that he would consider going to the emergency room.

In [59]:
"""Compute embeddings of the symptoms and save it to an Excel file"""
import cohere
import pandas as pd
import numpy as np

# Initialize cohere client with API key
co = cohere.ClientV2(api_key=COHERE_API_KEY)

# Load the Excel file
file_path = r'writst_symptoms.xlsx'
df = pd.read_excel(file_path)

# Extract the 'symptoms' column
symptoms = df['symptoms'].dropna().tolist()  # Remove nulls, if any

# Get the embeddings for each symptom
model = "embed-multilingual-v3.0"
input_type = "search_query"

res = co.embed(texts=symptoms,
               model=model,
               input_type=input_type,
               embedding_types=['float'])

print("Computing embeddings...")
# Convert embeddings to list of numpy arrays for saving
embeddings = [embedding for embedding in res.embeddings.float]

# Add embeddings as a new column to the dataframe
df = df[df['symptoms'].notna()]  # Filter rows without symptoms
df['embeddings'] = embeddings

# Save the updated dataframe back to Excel
output_file_path = r'wrist_symptoms_with_embeddings.xlsx'
df.to_excel(output_file_path, index=False)

print(f"Embeddings saved to {output_file_path}")


Computing embeddings...
Embeddings saved to C:\Users\h02317\Downloads\wrist_symptoms_with_embeddings.xlsx


In [95]:
"""Load the embedding files and match the user's input to find the matching symptom"""
import cohere
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.max_colwidth', None)  # Prevent truncating long text

# Initialize cohere client with API key
co = cohere.Client(api_key=COHERE_API_KEY)

# Load the Excel file with embeddings
file_path = r'wrist_symptoms_with_embeddings.xlsx'
df = pd.read_excel(file_path)

# Ensure embeddings are loaded as numpy arrays
df['embeddings'] = df['embeddings'].apply(lambda x: np.array(eval(x)))

# Function to get embeddings using the text-embedding model
def get_embedding(text, model="embed-multilingual-v3.0"):
    text = text.replace("\n", " ")
    response = co.embed(texts=[text], model=model, input_type="search_query", truncate="END")
    return np.array(response.embeddings[0])

def ask_and_suggest(input):
    # Ask the user for input
    user_input = input.strip()#input("Do you have pain in your wrist? ").strip()
    
    # Check if the user's input is not null
    if not user_input:
        print("You did not provide any input.")
    else:
        # Compute the embedding for the user's input
        user_embedding = get_embedding(user_input)
    
        # Compute cosine similarity between user's embedding and each row in the dataframe
        df['similarity'] = df['embeddings'].apply(lambda emb: cosine_similarity([user_embedding], [emb])[0][0])
    
        # Find the row with the maximum similarity score
        max_similarity_row = df.loc[df['similarity'].idxmax()]
    
        # Check if the maximum similarity score is above 0.30. This threshold can be changed. 
        if max_similarity_row['similarity'] > 0.30:
            print(f"User input: {user_input}\nMost similar symptom: {max_similarity_row['symptoms']} (Score: {max_similarity_row['similarity']:.2f})")
        else:
            print("No close match found with a similarity score above 0.30.")

###Some examples
print("Do you have pain in your wrist?")
print("\n")
ask_and_suggest("minulla on sarkee rannenivelta")
print("\n")
ask_and_suggest("I have a pain in the joints of my wrist")
print("\n")
ask_and_suggest("Ranteeniveleni on kipeä")
print("\n")
ask_and_suggest("Ranteeni nivel on kipeä")
print("\n")
ask_and_suggest("Jag har ont i handleden")

Do you have pain in your wrist?


User input: minulla on sarkee rannenivelta
Most similar symptom: särkee ranneniveltä (Score: 0.79)


User input: I have a pain in the joints of my wrist
Most similar symptom: wrist joint pain (Score: 0.88)


User input: Ranteeniveleni on kipeä
Most similar symptom: ranteessa kipua (Score: 0.84)


User input: Ranteeni nivel on kipeä
Most similar symptom: ranteessa kipua (Score: 0.83)


User input: Jag har ont i handleden
Most similar symptom: smärta i handleden (Score: 0.93)
